In [1]:
# Specify the path to your .log file
file_path = "C:\\Users\\AZA\\Desktop\\PFA\\DATA\\alert_FRET.log"


        
def read_log_file(file_path):
    with open(file_path, 'r') as file:
        log = file.readlines()
    return log


log = read_log_file(file_path)
log[:10]  # Afficher les 10 premières lignes pour comprendre la structure




['2023-10-31T00:22:56.097799+01:00\n',
 'Starting ORACLE instance (normal) (OS id: 2359880)\n',
 '2023-10-31T00:22:56.123085+01:00\n',
 '**********************************************************************\n',
 '2023-10-31T00:22:56.124890+01:00\n',
 'Dump of system resources acquired for SHARED GLOBAL AREA (SGA) \n',
 '\n',
 '2023-10-31T00:22:56.127019+01:00\n',
 ' Available system pagesizes:\n',
 '  4K, 64K, 16M, 16G \n']

In [25]:

import re

def classifier_log(log_entry):
    if re.search(r'ERROR|CRITICAL|ORA-\d{5}', log_entry, re.IGNORECASE):
        return 'rouge'
    elif re.search(r'WARN|WARNING', log_entry, re.IGNORECASE):
        return 'orange'
    elif re.search(r'NOTE|INFO|NOTICE', log_entry, re.IGNORECASE):
        return 'jaune'
    else:
        return 'vert'

def traiter_et_classifier(file_path):
    logs = read_log_file(file_path)
    resultats = []
    for log in logs:
        classification = classifier_log(log)
        resultats.append(f"{classification.upper()}: {log.strip()}")
    return resultats

# Classifier les logs
resultats = traiter_et_classifier(file_path)

# Afficher les  résultats
for resultat in resultats[100:1000]:
    print(resultat)




VERT: 2023-10-31T00:23:00.306050+01:00
VERT: ============================================================
JAUNE: NOTE: PatchLevel of this instance 0
VERT: ============================================================
VERT: Starting background process PMON
VERT: 2023-10-31T00:23:01.207538+01:00
VERT: PMON started with pid=2, OS id=5374638
VERT: 2023-10-31T00:23:01.210115+01:00
VERT: Starting background process CLMN
VERT: 2023-10-31T00:23:01.246968+01:00
VERT: CLMN started with pid=3, OS id=7144062
VERT: Starting background process PSP0
VERT: 2023-10-31T00:23:01.286442+01:00
VERT: PSP0 started with pid=4, OS id=14287534
VERT: Starting background process VKTM
VERT: 2023-10-31T00:23:01.336582+01:00
ROUGE: Errors in file /oracle/ora_base/diag/rdbms/fretdg/FRET/trace/FRET_vktm_5178052.trc  (incident=41):
ROUGE: ORA-00800: soft external error, arguments: [Set Priority Failed], [VKTM], [Check traces and OS configuration], [Check Oracle document and MOS notes], []
VERT: Incident details in: /ora

In [24]:
def classify_ALL_log(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()
    
    error_keywords = ['error', 'failed', 'critical']
    warning_keywords = ['warning', 'deprecated']

    error_count = 0
    warning_count = 0

    for line in lines:
        if any(keyword in line.lower() for keyword in error_keywords):
            error_count += 1
        elif any(keyword in line.lower() for keyword in warning_keywords):
            warning_count += 1

    if error_count > 100000:
        classification = 'Rouge - Problème critique détecté'
    elif warning_count > 100:
        classification = 'Jaune - Avertissements détectés'
    else:
        classification = 'Vert - Aucun problème détecté'

    return classification

# Utilisation de la fonction
classification = classify_ALL_log(file_path)
print(f'Classification du fichier de log : {classification}')

Classification du fichier de log : Vert - Aucun problème détecté


In [22]:
import re
from sklearn.feature_extraction.text import CountVectorizer

# obtenir la longueur de l'information
def get_len(info_comp):
    return len(info_comp)

# obtenir des symboles spécials pour logs SQL
def get_sql_char(info_comp):
    return len(re.findall("[?=\*#@%>]", info_comp, re.IGNORECASE))

# obtenir des mots spécials pour logs SQL
def get_sql_word(info_comp):
    return len(re.findall("(SELECT)|(version)|(comment)|(user)|(FROM)|(mysql)|(host)|(DELETE)|(name)|(MYSQL)|(WHERE)|(password)|(ORDER)",info_comp,re.IGNORECASE))

# obtenir des symboles spécials pour logs XSS
def get_xss_char(info_comp):
    return len(re.findall("[<>\"'=&{}!#\*]", info_comp, re.IGNORECASE))

# obtenir des mots spécials pour logs XSS
def get_xss_word(info_comp):
    return len(re.findall("(alert)|(script)|(XSS)|(SRC=)|(xss)|(onerror)|(onload)|(BODY)|(src=)|(INPUT)|(http)|(LINK)|(url=)|(SCRIPT)", info_comp, re.IGNORECASE))

# obtenir des motifs de chiffres courants dans les mots de passe
def get_password_number(info_comp):
    return len(re.findall("((?:0(?=1)|1(?=2)|2(?=3)|3(?=4)|4(?=5)|5(?=6)|6(?=7)|7(?=8)|8(?=9)|9(?=0)){2}\d)|((\d)\1{2})", info_comp, re.IGNORECASE))

# obtenir les caractéristiques combinées
def get_feature(info_comp):
    return [get_len(info_comp), get_sql_char(info_comp), get_sql_word(info_comp), get_xss_char(info_comp), get_xss_word(info_comp), get_password_number(info_comp)]

# Exemple d'utilisation
log_example = "SELECT * FROM users WHERE username='admin' AND password='12345'; <script>alert('XSS');</script>"

features = get_feature(log_example)
print(features)

KSJDHEDKE
